<a href="https://colab.research.google.com/github/sadhana62/AI/blob/master/Gauss_seidel_assignment_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### What is the Gauss–Seidel method?
Gauss–Seidel is an iterative method for solving Ax = b. Like Jacobi, it updates variable estimates repeatedly, but each time it computes a new value for a variable, it **immediately uses that new value** for subsequent updates in the same iteration. This in-place update usually makes Gauss–Seidel converge faster than Jacobi. It works well when A is diagonally dominant or when the iteration matrix has spectral radius < 1.


Step 1 — input the linear system

In [ ]:
n = int(input("Number of variables / equations (n): "))

print("Enter each row of A (n numbers) then the corresponding b value, separated by spaces.")
A = []
b = []
for i in range(n):
    parts = list(map(float, input(f"Row {i+1} (A_row then b): ").strip().split()))
    if len(parts) != n + 1:
        raise ValueError(f"Expected {n+1} numbers: {n} for A row and 1 for b.")
    A.append(parts[:n])
    b.append(parts[-1])

print("\nA =")
for row in A:
    print(row)
print("b =", b)


Number of variables / equations (n): 3
Enter each row of A (n numbers) then the corresponding b value, separated by spaces.
Row 1 (A_row then b): 4 -1 1 7
Row 2 (A_row then b): 1 3 -1 4
Row 3 (A_row then b): -2 1 5 6

A =
[4.0, -1.0, 1.0]
[1.0, 3.0, -1.0]
[-2.0, 1.0, 5.0]
b = [7.0, 4.0, 6.0]


Step 2 —  convergence check

In [ ]:
def is_strictly_diagonally_dominant(A):
    n = len(A)
    for i in range(n):
        diag = abs(A[i][i])
        off = sum(abs(A[i][j]) for j in range(n) if j != i)
        if diag <= off:
            return False
    return True

if is_strictly_diagonally_dominant(A):
    print("Matrix A is strictly diagonally dominant → Gauss–Seidel should converge.")
else:
    print("Matrix A is NOT strictly diagonally dominant. Gauss–Seidel may still converge, but not guaranteed.")


Matrix A is strictly diagonally dominant → Gauss–Seidel should converge.


Step 4 —  Implementing gauss seidel

In [ ]:
import math

def gauss_seidel(A, b, x0=None, tol=1e-8, maxiter=500):
    n = len(A)
    # initial guess
    if x0 is None:
        x = [0.0]*n
    else:
        x = x0[:]

    history = []
    for k in range(1, maxiter+1):
        for i in range(n):
            # compute sum of A[i][j]*x[j] for j != i
            s = 0.0
            for j in range(n):
                if j != i:
                    s += A[i][j] * x[j]   # note: x[j] uses newest values (Gauss-Seidel)
            if abs(A[i][i]) < 1e-16:
                raise ZeroDivisionError(f"Zero (or nearly zero) diagonal element A[{i}][{i}].")
            x[i] = (b[i] - s) / A[i][i]

        # compute residual r = A*x - b
        r = [sum(A[i][j]*x[j] for j in range(n)) - b[i] for i in range(n)]
        resnorm = math.sqrt(sum(ri*ri for ri in r))
        history.append((k, x[:], resnorm))

        if resnorm < tol:
            return x, history, True

    return x, history, False


Step - 4 printing the resulls

In [ ]:
x0 = None  # or provide initial guess like [0.0]*n or custom
solution, hist, converged = gauss_seidel(A, b, x0=x0, tol=1e-8, maxiter=500)

if converged:
    print(f"Gauss–Seidel converged in {len(hist)} iterations.")
else:
    print(f"Gauss–Seidel did NOT converge in {len(hist)} iterations (max reached).")

print("\nFirst 6 iterates (iter, x, residual-norm):")
for it, xvec, rnorm in hist[:6]:
    print(it, [round(v,12) for v in xvec], round(rnorm,12))

print("\nFinal approximate solution (last iterate):")
for i, val in enumerate(solution):
    print(f"x_{i} = {val}")


Gauss–Seidel converged in 20 iterations.

First 6 iterates (iter, x, residual-norm):
1 [1.75, 0.75, 1.75] 2.015564437075
2 [1.5, 1.416666666667, 1.516666666667] 0.929755045399
3 [1.725, 1.263888888889, 1.637222222222] 0.298738603275
4 [1.656666666667, 1.326851851852, 1.597296296296] 0.11036395706
5 [1.682388888889, 1.304969135802, 1.611961728395] 0.039380731731
6 [1.673251851852, 1.312903292181, 1.606720082305] 0.014180148957

Final approximate solution (last iterate):
x_0 = 1.67567567422231
x_1 = 1.310810812067541
x_2 = 1.608108107275416


Step 5 — final residual and interpretation

In [ ]:
n = len(A)
final_res = [sum(A[i][j]*solution[j] for j in range(n)) - b[i] for i in range(n)]
final_resnorm = math.sqrt(sum(r*r for r in final_res))
print("\nFinal residual norm:", final_resnorm)
if final_resnorm < 1e-8:
    print("Solution is accurate to the chosen tolerance.")
else:
    print("Residual is larger than tolerance; consider more iterations or check convergence criteria.")



Final residual norm: 8.507353081836538e-09
Solution is accurate to the chosen tolerance.
